## Step 1: Import Libraries

In [1]:
%run ../make_clean_names.py

In [2]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import logging

import polars as pl
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_session():
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries, pool_maxsize=10))
    return session

# Get API key from environment variables
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError("FMP_API_KEY not found in environment variables")

## Step 2: Import Symbols

In [3]:
def load_symbols(file_path: str) -> List[str]:
    """Load symbols from a text file"""
    try:
        with open(file_path, 'r') as f:
            symbols = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(symbols)} symbols from {file_path}")
        return symbols
    except Exception as e:
        print(f"Error loading symbols: {str(e)}")
        return []

symbols_file = '../tickers.txt'
symbols = load_symbols(symbols_file)

if symbols:
    print("Symbols:", symbols)
else:
    print("No symbols loaded.")

Loaded 61 symbols from ../tickers.txt
Symbols: ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'META', 'NVDA', 'AVGO', 'ORCL', 'CRM', 'ACN', 'ADBE', 'CSCO', 'INTC', 'NFLX', 'DIS', 'CMCSA', 'VZ', 'T', 'AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'LOW', 'WMT', 'PG', 'KO', 'PEP', 'COST', 'BRK-B', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'LLY', 'CAT', 'BA', 'HON', 'UPS', 'RTX', 'GE', 'XOM', 'CVX', 'COP', 'SLB', 'LIN', 'APD', 'ECL', 'PLD', 'AMT', 'CCI', 'OSW']


## Step 3: Extract Data from FMP into Polars

In [4]:
def fetch_dcf(symbol: str, api_key: str, session: requests.Session) -> Dict:
    """Fetch company profile data from FMP API"""
    url = f"https://financialmodelingprep.com/api/v3/discounted-cash-flow/{symbol}"
    params = {"apikey": api_key}
    
    try:
        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching {symbol}: {str(e)}")
        return None

def fetch_all_dcf(symbols: List[str], api_key: str) -> List[Dict]:
    session = create_session()
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [
            executor.submit(fetch_dcf, symbol, api_key, session)
            for symbol in symbols
        ]
        results = [
            f.result() for f in concurrent.futures.as_completed(futures)
        ]
    
    return [r[0] for r in results if r and isinstance(r, list)]

# Execute fetching
company_profiles = fetch_all_dcf(symbols, FMP_API_KEY)

# Convert to Polars DataFrame more efficiently
if company_profiles:
    df_dcf = pl.DataFrame(company_profiles)

In [5]:
def fetch_dcf_advanced(symbol: str, api_key: str, session: requests.Session) -> Dict:
    """Fetch company profile data from FMP API"""
    url = f"https://financialmodelingprep.com/api/v4/advanced_discounted_cash_flow"
    params = {
        "symbol": symbol,
        "apikey": api_key}
    
    try:
        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching {symbol}: {str(e)}")
        return None

def fetch_all_dcf_advanced(symbols: List[str], api_key: str) -> List[Dict]:
    session = create_session()
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [
            executor.submit(fetch_dcf_advanced, symbol, api_key, session)
            for symbol in symbols
        ]
        results = [
            f.result() for f in concurrent.futures.as_completed(futures)
        ]
    
    return [r[0] for r in results if r and isinstance(r, list)]

# Execute fetching
company_profiles = fetch_all_dcf_advanced(symbols, FMP_API_KEY)

# Convert to Polars DataFrame more efficiently
if company_profiles:
    df_dcf_advanced = pl.DataFrame(company_profiles)

## Step 4: Clean Column Names

In [6]:
df = df_dcf.join(df_dcf_advanced, on="symbol", how="left")

In [7]:
df = make_clean_names(df)

## Step 5: Write Polars to Parquet

In [8]:
output_dir = "../../../data/finance"

# Write DataFrame to Parquet
df.write_parquet(f'{output_dir}/company_dfc_advanced.parquet')

## Step 6: Read Parquet (Validate)

In [9]:
pl.scan_parquet(f'{output_dir}/company_dfc_advanced.parquet').head().collect()

symbol,date,dcf,stock_price,year,revenue,revenue_percentage,ebitda,ebitda_percentage,ebit,ebit_percentage,depreciation,depreciation_percentage,total_cash,total_cash_percentage,receivables,receivables_percentage,inventories,inventories_percentage,payable,payable_percentage,capital_expenditure,capital_expenditure_percentage,price,beta,diluted_shares_outstanding,costof_debt,tax_rate,after_tax_cost_of_debt,risk_free_rate,market_risk_premium,cost_of_equity,total_debt,total_equity,total_capital,debt_weighting,equity_weighting,wacc,tax_rate_cash,ebiat,ufcf,sum_pv_ufcf,long_term_growth_rate,terminal_value,present_terminal_value,enterprise_value,net_debt,equity_value,equity_value_per_share,free_cash_flowt1
str,str,f64,f64,str,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,f64,f64,i64,f64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,f64,i64
"""NVDA""","""2025-01-20""",56.938037,137.71,"""2029""",520683174055,53.59,195855645765,37.62,172283561763,33.09,23572084002,4.53,353940947259,67.98,80832177311,15.52,5.9766e10,11.48,29846344169,5.73,-24388759055,-4.68,137.71,1.657,24940000000,4.42,12.0,3.89,4.42,4.72,12.24,11056000000,3434487400000,3445543400000,0.32088,99.68,12.22,4301152,164873384699,125414004138,206841874168,4.0,1587647744761,892263483222,1099105357390,3776000000,1095329357390,43.92,130430564304
"""AAPL""","""2025-01-20""",145.517176,229.98,"""2029""",631734147383,10.07,203936738085,32.28,183897900530,29.11,20038837554,3.17,120316909203,19.05,96053677011,15.2,1.0168e10,1.61,101845034847,16.12,-17288478562,-2.74,229.98,1.24,15408095000,4.42,24.09,3.36,4.42,4.72,10.27,106629000000,3543553688100,3650182688100,2.92,97.08,10.08,16549050,153464546253,155814583110,483770706269,4.0,2665047548540,1648746062304,2132516768573,76686000000,2055830768573,133.43,162047166435
"""GOOGL""","""2025-01-20""",193.519212,196.0,"""2028""",706109768075,18.1,240199708049,34.02,200287429124,28.36,39912278924,5.65,394484923413,55.87,111949781484,15.85,3.4094e9,0.48284,18488261260,2.62,-81828362399,-11.59,196.0,1.034,12722000000,4.42,13.91,3.81,4.42,4.72,9.3,28504000000,2493512000000,2522016000000,1.13,98.87,9.24,15122195,169999573836,113239806868,311784227902,4.0,2248466149772,1445513996197,1757298224099,4456000000,1752842224099,137.78,117769399143
"""META""","""2025-01-20""",448.364852,612.77,"""2028""",313052449124,18.34,138686926174,44.3,114523353776,36.58,24163572398,7.72,171414206869,54.76,38872999762,12.42,0.0,0.0,9274005045,2.96,-63771558124,-20.37,612.77,1.215,2629000000,4.42,17.56,3.64,4.42,4.72,10.15,37234000000,1610972330000,1648206330000,2.26,97.74,10.01,18291174,93575688865,49381215107,131846042860,4.0,854945977607,530685709913,662531752773,-4628000000,667159752773,253.77,51356463711
"""AVGO""","""2025-01-20""",177.097239,237.44,"""2029""",139008851482,21.93,72607157198,52.23,45906526818,33.02,26700630380,19.21,47633557276,34.27,12076255889,8.69,6.5147e9,4.69,4743635680,3.41,-1988799375,-1.43,237.44,1.187,4778000000,5.62,40.55,3.34,4.42,4.72,10.02,67566000000,1134488320000,1202054320000,5.62,94.38,9.74,11050874,40833454730,63054429826,161670081048,4.0,1142414858598,717787035187,879457116236,58218000000,821239116236,171.88,65576607019
